## Imporing Libraries

In [1]:
import  pandas as pd 
import numpy as np
import fuzzywuzzy
from fuzzywuzzy import process
import chardet
import warnings 
warnings.filterwarnings('ignore')

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Reading Dataset

In [2]:
professors=pd.read_csv("pakistan_intellectual_capital.csv")

In [3]:
professors.head()

,Unnamed: 0,S#,Teacher Name,University Currently Teaching,Department,Province University Located,Designation,Terminal Degree,Graduated from,Country,Year,Area of Specialization/Research Interests,Other Information
0,2,3,Dr. Abdul Basit,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,Software Engineering & DBMS,NaN
1,4,5,Dr. Waheed Noor,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,DBMS,NaN
2,5,6,Dr. Junaid Baber,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"Information processing, Multimedia mining",NaN
3,6,7,Dr. Maheen Bakhtyar,University of Balochistan,Computer Science & IT,Balochistan,Assistant Professor,PhD,Asian Institute of Technology,Thailand,NaN,"NLP, Information Retrieval, Question Answering...",NaN
4,24,25,Samina Azim,Sardar Bahadur Khan Women's University,Computer Science,Balochistan,Lecturer,BS,Balochistan University of Information Technolo...,Pakistan,2005.0,VLSI Electronics DLD Database,NaN


In [4]:
# Cheking for Unique Values For All Columns


for i in professors.columns:
    print(f"{i}\n{professors[i].unique()}\n\n")
    
    

Unnamed: 0
[   2    4    5 ... 1977 1979 1980]


S#
[   3    5    6 ... 1978 1980 1981]


Teacher Name
['Dr. Abdul Basit' 'Dr. Waheed Noor' 'Dr. Junaid Baber' ...
 'Dr. Rashad M Jillani' 'Dr. Shahabuddin Ansari' 'Dr. Sajid Anwar']


University Currently Teaching
['University of Balochistan' "Sardar Bahadur Khan Women's University"
 'University of Turbat' 'COMSATS, Islamabad Campus'
 'National University of Sciences and Technology' 'RIPHAH International'
 'FAST-NU(Islamabad)' 'International Islamic University,Islamabad'
 'National University of Modern Languages' 'Air University'
 'Bahria University,Islamabad'
 'Capital University of Science and Technology'
 'Pakistan Institute of Engineering and Applied Sciences'
 'University of Sargodha,Mandi Bahauddin Campus' 'NAML-Mianwali'
 'University of Lahore-PakPattan' 'University of Sahiwal'
 'Barani Institute of Information and Technology'
 'Fatima Jinnah Women University' 'FAST(Faisalabad)'
 'University of Central Punjab' 'Lahore Garrison Uni

In [5]:
# Lowering Case of String
professors['Country']=professors['Country'].str.lower()

# Removing White Space
professors['Country']=professors['Country'].str.strip()

In [6]:
countries=professors['Country'].unique()
countries.sort()
countries

array(['australia', 'austria', 'canada', 'china', 'finland', 'france',
       'germany', 'greece', 'hongkong', 'ireland', 'italy', 'japan',
       'macau', 'malaysia', 'mauritius', 'netherland', 'new zealand',
       'norway', 'pakistan', 'portugal', 'russian federation',
       'saudi arabia', 'scotland', 'singapore', 'south korea',
       'southkorea', 'spain', 'sweden', 'thailand', 'turkey', 'uk',
       'urbana', 'usa', 'usofa'], dtype=object)

In [7]:
#Getiing Matches for South Korea
matches=fuzzywuzzy.process.extract('south korea',countries,limit=10,
                                   scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('south korea', 100),
 ('southkorea', 48),
 ('saudi arabia', 43),
 ('norway', 35),
 ('ireland', 33),
 ('portugal', 32),
 ('singapore', 30),
 ('netherland', 29),
 ('macau', 25),
 ('usofa', 25)]

In [8]:
# Function For Replacing And Finding Matches

def replace_matches_in_column(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    print(f"\nmatches\n{matches}\n")
    
    # only get matches with a ratio > min_ratio
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]
    print(f"\n\nclose matches\n{close_matches}\n")
    
    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)
    print(f"\n\nrows with match\n {rows_with_matches}\n")
    
    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [9]:
# Calling Function To Replace The Value of South Korea

replace_matches_in_column(professors,'Country','south korea',min_ratio=47)



matches
[('south korea', 100), ('southkorea', 48), ('saudi arabia', 43), ('norway', 35), ('ireland', 33), ('portugal', 32), ('singapore', 30), ('netherland', 29), ('usofa', 25), ('macau', 25)]



close matches
['south korea', 'southkorea']



rows with match
 0       False
1       False
2       False
3       False
4       False
        ...  
1137     True
1138    False
1139    False
1140    False
1141     True
Name: Country, Length: 1142, dtype: bool

All done!


In [10]:
#Getiing Matches for usa
matches=fuzzywuzzy.process.extract('usa',countries,limit=10,
                                   scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('usa', 100),
 ('usofa', 75),
 ('austria', 60),
 ('australia', 50),
 ('spain', 50),
 ('urbana', 44),
 ('uk', 40),
 ('malaysia', 36),
 ('pakistan', 36),
 ('portugal', 36)]

In [11]:
# Calling Function To Replace The Value of usa

replace_matches_in_column(professors,'Country','usa',min_ratio=74)



matches
[('usa', 100), ('usofa', 75), ('austria', 60), ('australia', 50), ('spain', 50), ('urbana', 44), ('uk', 40), ('pakistan', 36), ('malaysia', 36), ('portugal', 36)]



close matches
['usa', 'usofa']



rows with match
 0       False
1       False
2       False
3       False
4       False
        ...  
1137    False
1138    False
1139     True
1140    False
1141    False
Name: Country, Length: 1142, dtype: bool

All done!


### For Department Column

In [12]:
# Lowering the Case of Department Column
professors['Department']=professors['Department'].str.lower()

#Removing White Space from String 
professors['Department']=professors['Department'].str.strip()



In [13]:
# getting unique of Department Column
deparments=professors['Department'].unique()
deparments.sort()
deparments

array(['computer and information sciences', 'computer engineering',
       'computer science', 'computer science & it',
       'computer science and engineering', 'computer science and it',
       'computer science and software engineering', 'computer sciences',
       'computing', 'computing & information sciences', 'cs & it',
       'engineering, science & technology',
       'faculty of computing & engineering', 'information technology',
       'institute of mathematics & computer science',
       'school of information and technology', 'software engineering'],
      dtype=object)

In [14]:
#Getting Matches of Department Column
matches=fuzzywuzzy.process.extract('computer science & it',deparments,limit=10,
                                   scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('computer science & it', 100),
 ('computer science', 91),
 ('computer science and it', 90),
 ('computer sciences', 89),
 ('computer and information sciences', 73),
 ('computer science and engineering', 71),
 ('computing & information sciences', 69),
 ('institute of mathematics & computer science', 63),
 ('computer engineering', 62),
 ('computer science and software engineering', 60)]

In [15]:
#defining Function for Exact match of Ratio
def replace_matches_in_column1(df, column, string_to_match, min_ratio = 47):
    # get a list of unique strings
    strings = df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    print(f"\nmatches\n{matches}\n")
    
    # only get matches with a ratio > min_ratio
    close_matches = [matches[0] for matches in matches if matches[1] == min_ratio]
    print(f"\n\nclose matches\n{close_matches}\n")
    
    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)
    print(f"\n\nrows with match\n {rows_with_matches}\n")
    
    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # let us know the function's done
    print("All done!")

In [16]:
replace_matches_in_column1(professors,"Department",'computer science & it',90)


matches
[('computer science & it', 100), ('computer science', 91), ('computer science and it', 90), ('computer sciences', 89), ('computer and information sciences', 73), ('computer science and engineering', 71), ('computing & information sciences', 69), ('institute of mathematics & computer science', 63), ('computer engineering', 62), ('computer science and software engineering', 60)]



close matches
['computer science and it']



rows with match
 0       False
1       False
2       False
3       False
4       False
        ...  
1137    False
1138    False
1139    False
1140    False
1141    False
Name: Department, Length: 1142, dtype: bool

All done!


### Repeating Same Procedure for other Values

In [17]:
matches=fuzzywuzzy.process.extract('computer science',deparments,limit=10,
                                   scorer=fuzzywuzzy.fuzz.token_sort_ratio)
matches

[('computer science', 100),
 ('computer sciences', 97),
 ('computer science & it', 91),
 ('computer science and it', 82),
 ('computer engineering', 67),
 ('computer science and engineering', 67),
 ('computer and information sciences', 65),
 ('computing & information sciences', 65),
 ('computing', 64),
 ('computer science and software engineering', 56)]

In [18]:
replace_matches_in_column1(professors,"Department",'computer science',97)


matches
[('computer science', 100), ('computer sciences', 97), ('computer science & it', 91), ('computer engineering', 67), ('computer science and engineering', 67), ('computer and information sciences', 65), ('computing & information sciences', 65), ('computing', 64), ('computer science and software engineering', 56), ('institute of mathematics & computer science', 56)]



close matches
['computer sciences']



rows with match
 0       False
1       False
2       False
3       False
4       False
        ...  
1137    False
1138    False
1139    False
1140    False
1141    False
Name: Department, Length: 1142, dtype: bool

All done!


In [19]:
professors['Department'].unique()

array(['computer science & it', 'computer science', 'computing',
       'computer science and software engineering',
       'computer and information sciences', 'software engineering',
       'information technology', 'cs & it',
       'school of information and technology',
       'institute of mathematics & computer science',
       'engineering, science & technology', 'computer engineering',
       'computing & information sciences',
       'faculty of computing & engineering',
       'computer science and engineering'], dtype=object)